# Ejemplo de un dashboard generado en Google colab

In [1]:
!pip install dash
!pip install jupyter_dash
!pip install dash_bootstrap_components
!pip install dash_daq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.2/221.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.7/642.7 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dash_daq: filename=dash_daq-0.5.0-py3-none-any.whl size=669691 sha256=f8377fb8fd8554141049dd945c7e4cbdaa8e672e8a96292caf76c0290b264801
  Stored in directory: /root/.cache/pip/wheels/75/14/1b/208d09d5e239391048bdc167759977b41ba65a3d4063aebf6b
Successfully built dash_daq


In [4]:
import pandas as pd #manipulacion
import numpy as np # caclulo matematicos en arrays
import math # libreria matematica

from sklearn.ensemble import RandomForestRegressor # libreria ml con randomforest para regresion


#librerias de graficos
import plotly.graph_objects as go #libreria avanzada de grafico

import dash # librearia de dashboard
from jupyter_dash import JupyterDash #joinea de dashboard con jupyter notebooks

import dash_core_components as dcc # libraria de componentes basicos para generar el dash,  esquelto - backend
import dash_bootstrap_components as dbc # componentes graficos symbolos, etc
import dash_html_components as html # libraria para escribir pagina web
import dash_daq as daq # libraria que trae todos los widgets ejemplo sliders
from dash.dependencies import Input, Output




df = pd.read_csv('/content/ejemplo_dash (3).csv')


In [5]:
df.head()

,Temperature,Viscosity,Pressure,pH,Inlet_flow,Rotating_Speed,Particles_size,Color_density,Y
0,20.202146,0.322192,10.356239,6.548957,-0.827285,-1.126331,-0.671913,-0.709468,89.080483
1,18.732836,0.650678,9.384076,6.204418,-0.383894,-0.374792,0.825484,-0.653890,77.718641
2,21.722734,0.276771,10.221257,6.951655,0.326087,0.392589,2.684909,2.272731,113.800527
3,21.015227,-0.052914,10.298739,6.197527,0.824042,0.663623,-0.087576,-0.036282,93.457770
4,18.703859,0.084677,8.157001,6.542089,-0.132860,-0.638585,-0.372161,1.095319,74.851481


In [6]:
# entenamos un randomforest
#1°traer modelo
model = RandomForestRegressor()
#2°divimos entre x e y
x_values =df.drop("Y", axis=1)
target_values = df["Y"]
#3°splitear los datos
#x_train, x_test, y_train, y_test = train_test_split(x_values,target_values , test_size=0.2, random_state=42)
#4°fiteo entrenamos la datos
model.fit(x_values, target_values)#x_train, y_train
#5°predicciones
#y_pred_test = modelo_lineal.predict(x_test)
#y_pred_train = modelo_lineal.predict(x_train)

RandomForestRegressor()

In [7]:
#creamos un dataframe con las variables importantes
df_feature_importances = pd.DataFrame(model.feature_importances_*100,columns=["Importance"],index=list(df.drop("Y", axis=1).columns))
df_feature_importances = df_feature_importances.sort_values("Importance", ascending=False)
df_feature_importances

,Importance
Pressure,30.421732
Temperature,29.382313
pH,19.666993
Inlet_flow,10.090624
Particles_size,7.151263
Viscosity,1.120777
Rotating_Speed,1.119290
Color_density,1.047008


In [8]:
# creamos un plot con las variables importants
fig_features_importance = go.Figure()
fig_features_importance.add_trace(go.Bar(x=df_feature_importances.index,
                                         y=df_feature_importances["Importance"],
                                         marker_color='rgb(171, 226, 251)')
                                 )
fig_features_importance.update_layout(title_text='<b>Features Importance of the model<b>', title_x=0.5)


########################


# guardamos min,  max , promedio de loas 3 variables mas importants
dropdown_1_label = df_feature_importances.index[0]
dropdown_1_min = math.floor(df[dropdown_1_label].min())
dropdown_1_mean = round(df[dropdown_1_label].mean())
dropdown_1_max = round(df[dropdown_1_label].max())

dropdown_2_label = df_feature_importances.index[1]
dropdown_2_min = math.floor(df[dropdown_2_label].min())
dropdown_2_mean = round(df[dropdown_2_label].mean())
dropdown_2_max = round(df[dropdown_2_label].max())

dropdown_3_label = df_feature_importances.index[2]
dropdown_3_min = math.floor(df[dropdown_3_label].min())
dropdown_3_mean = round(df[dropdown_3_label].mean())
dropdown_3_max = round(df[dropdown_3_label].max())

In [12]:
app = JupyterDash(__name__)
server = app.server

# The page structure will be:
#    Features Importance Chart
#    <H4> Feature #1 name
#    Slider to update Feature #1 value
#    <H4> Feature #2 name
#    Slider to update Feature #2 value
#    <H4> Feature #3 name
#    Slider to update Feature #3 value
#    <H2> Updated Prediction
#    Callback fuction with Sliders values as inputs and Prediction as Output

# HTML Layout
app.layout = html.Div(style={'textAlign': 'center', 'width': '800px', 'font-family': 'Verdana'},

                    children=[

                        # primer layout - titulo
                        html.H1(children="Simulation Tool"),

                        #vinculado a grafico del primer layout
                        dcc.Graph(figure=fig_features_importance),

                        # Display la variable mas importante
                        html.H4(children=dropdown_1_label),

                        # Slider de la varailbes iporatnte
                        dcc.Slider(
                            id='X1_slider',
                            min=dropdown_1_min,
                            max=dropdown_1_max,
                            step=0.5,
                            value=dropdown_1_mean,
                            marks={i: '{} bars'.format(i) for i in range(dropdown_1_min, dropdown_1_max+1)}
                            ),

                        # Varaible 2
                        html.H4(children=dropdown_2_label),

                        dcc.Slider(
                            id='X2_slider',
                            min=dropdown_2_min,
                            max=dropdown_2_max,
                            step=0.5,
                            value=dropdown_2_mean,
                            marks={i: '{}°'.format(i) for i in range(dropdown_2_min, dropdown_2_max+1)}
                        ),
                        #variable 3
                        html.H4(children=dropdown_3_label),

                        dcc.Slider(
                            id='X3_slider',
                            min=dropdown_3_min,
                            max=dropdown_3_max,
                            step=0.1,
                            value=dropdown_3_mean,
                            marks={i: '{}'.format(i) for i in np.linspace(dropdown_3_min,dropdown_3_max,1+(dropdown_3_max-dropdown_3_min)*5)},
                        ),

                        # Valor de la prediccion
                        html.H2(id="prediction_result"),

                    ])

# esto es un manera de poder devolver el valor de una prediccion o una funcion
@app.callback(Output(component_id="prediction_result",component_property="children"),
              #valores a tomar en cuenta en la salida
              [Input("X1_slider","value"), Input("X2_slider","value"), Input("X3_slider","value")])

#calculo de la predicion reacitva
def update_prediction(X1, X2, X3):

    input_X = np.array([X1,
                       df["Viscosity"].mean(),
                       df["Particles_size"].mean(),
                       X2,
                       df["Inlet_flow"].mean(),
                       df["Rotating_Speed"].mean(),
                       X3,
                       df["Color_density"].mean()]).reshape(1,-1)

    # Prediction is calculated based on the input_X array
    prediction = model.predict(input_X)[0]

    # And retuned to the Output of the callback function
    return "Prediction: {}".format(round(prediction,1))


app.run_server(mode='external')

/usr/local/lib/python3.10/dist-packages/dash/dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

Dash app running on:


<IPython.core.display.Javascript object>